In [1]:
import os
teams = ["MIA", "MIL", "BOS", "BRK", "TOR", "PHI", "NYK", "CHI", "CLE", "DET", "IND", "WAS", "ATL", "ORL", "CHO", "MIN", "OKC", "DEN", 
"POR", "UTA", "LAL", "LAC", "GSW", "PHO", "MEM", "SAC", "NOP", "HOU", "SAS", "DAL"]
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time
import asyncio
import requests


In [2]:
rosters = {}

for team in teams:
    url = f"https://www.basketball-reference.com/teams/{team}/2024.html"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    
    roster_table = soup.find("table", {"id": "roster"})
    players = []

    if roster_table:
        rows = roster_table.find_all("tr")
        for row in rows[1:]:
            player_name = row.find("td", {"data-stat": "player"}).text
            cleaned_name = player_name.replace("\xa0\xa0(TW)", "").strip()
            players.append(cleaned_name)
    
    rosters[team] = players

print(rosters)

{'MIA': ['Thomas Bryant', 'Alondes Williams', 'Justin Champagnie', 'Drew Peterson', 'Caleb Daniels', 'Cole Swider', 'Jamal Cain', 'Kevin Love', 'Jamaree Bouyea', 'Jaime Jaquez Jr.', 'Orlando Robinson', 'Bam Adebayo', 'Josh Richardson', 'Dru Smith', 'Tyler Herro', 'Caleb Martin', 'Nikola Jović', 'Haywood Highsmith', 'Jimmy Butler', 'Duncan Robinson', 'Kyle Lowry'], 'MIL': ['Khris Middleton', 'Marques Bolden', 'TyTy Washington Jr.', 'Thanasis Antetokounmpo', 'Jazian Gortman', 'Jae Crowder', 'Chris Livingston', 'A.J. Green', 'Andre Jackson Jr.', 'Giannis Antetokounmpo', 'Brook Lopez', 'Malik Beasley', 'Robin Lopez', 'Pat Connaughton', 'MarJon Beauchamp', 'Bobby Portis', 'Grayson Allen', 'Jrue Holiday'], 'BOS': ['Al Horford', 'Neemias Queta', 'Taylor Funk', 'Svi Mykhailiuk', 'Jay Scrubb', 'Dalano Banton', 'JD Davison', 'Jordan Walsh', 'Oshae Brissett', 'Kristaps Porziņģis', 'Sam Hauser', 'Luke Kornet', 'Malcolm Brogdon', 'Robert Williams', 'Derrick White', 'Jayson Tatum', 'Payton Pritchard

In [3]:
import pygame
import random
import os
from unidecode import unidecode
import pygame_menu
import sys

# Initialize Pygame
pygame.init()

# Set up the screen
screen_width = 800
screen_height = 600
screen = pygame.display.set_mode((screen_width, screen_height))
pygame.display.set_caption("NBA Roster Guessing Game")

# Create a font
font = pygame.font.Font(None, 36)

# Define the teams and players (add your rosters dictionary here)
teams = list(rosters.keys())
all_players = [player for team_roster in rosters.values() for player in team_roster]
correct_boxes = [False] * len(all_players)

# Choose a random team and prepare logos
random_team = random.choice(teams)
random_roster = rosters[random_team]
logo_folder = "Pictures"
team_logos = {}
for team in rosters:
    logo_filename = os.path.join(logo_folder, f"{team}.png")
    if os.path.exists(logo_filename):
        team_logos[team] = pygame.image.load(logo_filename)
        team_logos[team] = pygame.transform.scale(team_logos[team], (100, 100))

# Define player box dimensions
player_box_width = 250
player_box_height = 40
player_box_margin_x = 10
player_box_margin_y = 10
player_box_start_x = 10
player_box_start_y = 250

# Initialize current team and roster
current_team_index = teams.index(random.choice(teams))
current_team = teams[current_team_index]
current_roster = rosters[current_team]

# Create player boxes based on the current roster length
player_boxes = [
    pygame.Rect(player_box_start_x + (i % 3) * (player_box_width + player_box_margin_x),
                player_box_start_y + (i // 3) * (player_box_height + player_box_margin_y),
                player_box_width, player_box_height)
    for i in range(len(current_roster))
]

# Set up a smaller font and game variables
small_font = pygame.font.Font(None, 30)
running = True
guessed_players = []
search_text = ""
input_text = ""
correct_guess = False
all_players_lower = [unidecode(player.lower()) for player in all_players]
suggested_players = []
correct_guesses = sum(correct_boxes)
total_players = len(current_roster)
strikes = 0
max_strikes = 3

# Set up the image for strikes
pygame.display.set_caption("NBA Roster Guessing Game")
image = pygame.image.load("strike.jpeg")  # Replace with the path to your image
image = pygame.transform.scale(image, (40, 40))  # Scale the image to your desired size
image_rect = image.get_rect()

# Timer settings
image_display_duration = 120  # Duration in frames
remaining_display_frames = image_display_duration

# Main game loop
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_RETURN:
                if search_text:
                    cleaned_search = unidecode(search_text.lower())
                    is_correct_guess = False
                    for i, player in enumerate(current_roster):
                        if cleaned_search == unidecode(player.lower()) and search_text not in guessed_players:
                            correct_boxes[i] = True
                            is_correct_guess = True
                            break  # Exit the loop when a correct guess is found
            elif event.key == pygame.K_BACKSPACE:
                search_text = search_text[:-1]
            elif event.type == pygame.MOUSEBUTTONUP:  # Left mouse button clicked
                mouse_x, mouse_y = event.pos
                if suggestion_rect.collidepoint(mouse_x, mouse_y):
                    if suggested_player in current_roster:
                        search_text = suggested_player
                        guessed_players.append(search_text)
                        correct_boxes[current_roster.index(search_text)] = True
                    else:
                        search_text = ""
            else:
                search_text += event.unicode

    # Filter suggested players based on the search text
    suggested_players = [
        player for player in all_players
        if unidecode(search_text.lower()) in unidecode(player.lower()) and player not in guessed_players
    ]

    # Update the screen
    screen.fill((255, 255, 255))

    # Display strikes for incorrect guesses
    if remaining_display_frames > 0 and strikes > 0:
        for i in range(strikes):
            screen.blit(image, (400 + i * 50, 30))

    # Reset the game if max strikes reached
    if strikes == max_strikes:
        current_team_index = teams.index(random.choice(teams))
        current_team = teams[current_team_index]
        current_roster = rosters[current_team]
        correct_boxes = [False] * len(all_players)
        guessed_players = []
        strikes = 0
        correct_guesses = 0
        total_players = len(current_roster)

    # Display current team's logo
    current_logo = team_logos[current_team]
    screen.blit(current_logo, (650, 50))


    # Create and display the search bar
    search_bar = pygame.Surface((300, 40))
    search_bar.fill((200, 200, 200))
    pygame.draw.rect(search_bar, (0, 0, 0), (0, 0, 300, 40), 2)
    screen.blit(search_bar, (50, 30))

    # Display the search text again
    search_text_surface = font.render(search_text, True, (0, 0, 0))
    screen.blit(search_text_surface, (50, 30))

    max_suggestions = 5

    # Display current score
    score_text = f"Score: {correct_guesses}/{total_players}"
    score_surface = font.render(score_text, True, (0, 0, 0))
    screen.blit(score_surface, (screen_width - score_surface.get_width() - 20, 20))

    # Handling guesses
    for i, (player_box, is_correct) in enumerate(zip(player_boxes, correct_boxes)):
        if i < len(current_roster):
            player_name = current_roster[i]
            pygame.draw.rect(screen, (200, 200, 200), player_box)
            pygame.draw.rect(screen, (0, 0, 0), player_box, 2)

            if len(search_text) >= 3:
                for i, suggested_player in enumerate(suggested_players[:max_suggestions]):
                    suggestion_surface = small_font.render(suggested_player, True, (0, 0, 0))
                    suggestion_x = 50
                    suggestion_y = 70 + i * 30
                    suggestion_rect = pygame.Rect(suggestion_x, suggestion_y, 250, 30)

                    # Check if mouse is hovering over the suggestion box
                    mouse_x, mouse_y = pygame.mouse.get_pos()
                    if suggestion_rect.collidepoint(mouse_x, mouse_y):
                        pygame.draw.rect(screen, (0, 0, 255), suggestion_rect, 2)
                        if event.type == pygame.MOUSEBUTTONUP and event.button == 1:
                            search_text = suggested_player
                            guessed_players.append(search_text)
                            if suggested_player in current_roster:
                                correct_boxes[current_roster.index(search_text)] = True
                                pygame.event.post(pygame.event.Event(pygame.KEYDOWN, {'key': pygame.K_RETURN}))
                            if search_text not in current_roster:
                                strikes += 1
                                print(strikes)
                                search_text = ""
                            search_text = ""
                    else:
                        pygame.draw.rect(screen, (0, 0, 0), suggestion_rect, 2)

                    # Draw the suggestion text
                    screen.blit(suggestion_surface, (suggestion_x + 5, suggestion_y + 5))

            else:
                suggested_players = []

            if is_correct:
                pygame.draw.rect(screen, (0, 255, 0), player_box)
                player_name_text = small_font.render(player_name, True, (0, 0, 0))
                text_rect = player_name_text.get_rect(center=player_box.center)
                screen.blit(player_name_text, text_rect.topleft)
                correct_guesses = sum(correct_boxes)

            if unidecode(search_text.lower()) == unidecode(player_name.lower()) and is_correct:
                pygame.draw.rect(screen, (0, 255, 0), player_box, 0)  # Green highlighting

    pygame.display.flip()

pygame.quit()

if __name__ == "__main__":
    pygame.quit()




pygame 2.4.0 (SDL 2.26.4, Python 3.10.7)
Hello from the pygame community. https://www.pygame.org/contribute.html
pygame-menu 4.4.3
1
